In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing dataset 
tr=pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
tr.head()
# /kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns
%matplotlib inline

In [ ]:
tr.shape

In [ ]:
tr.info()

In [ ]:
# columns with object datatypes cant be taken forward in algorithm

In [ ]:
tr.drop(['enrollee_id','city'],axis=1,inplace=True)

In [ ]:
tr.isnull().mean()*100

In [ ]:
tr.dropna(subset=['enrolled_university','education_level','last_new_job'],axis=0,inplace=True)
# dropped null values as their mean is not high 

In [ ]:
y=tr['target']
y.astype(int)

In [ ]:
tr.isnull().mean()*100

In [ ]:
from sklearn.impute import SimpleImputer

impute=SimpleImputer(strategy='most_frequent')
x_imp=impute.fit_transform(tr)
x_imp

#  SimpleImputer is a scikit-learn class which is helpful in handling the missing data in the
#  predictive model dataset.It replaces the NaN values with a specified placeholder
#  and here we have used 'most_frequent' method.

In [ ]:
X=pd.DataFrame(x_imp,columns=tr.columns,index=tr.index)
X
#data was in array so we need to make DataFrame with same indexes and column names

In [ ]:
X.isnull().mean()
# now there is no null value

In [ ]:
X['training_hours']=X['training_hours'].astype(int)
# handling of object dtype 

In [ ]:
sns.distplot(X['training_hours'])
# plot for visualisation (distribution plot) as the data spread was high

In [ ]:
X['city_development_index']=X['city_development_index'].astype(float)
# handling of object dtype 

In [ ]:
X.info()


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Sklearn provides a very efficient tool for encoding the levels of categorical features
# into numeric values. LabelEncoder encode labels with a value between 0 and n_classes-1 
# where n is the number of distinct labels # If a label repeats it assigns the same value
# to as assigned earlier.

In [ ]:
X['gender']=LabelEncoder().fit_transform(X['gender'])
X['relevent_experience']=LabelEncoder().fit_transform(X['relevent_experience'])
X['enrolled_university']=LabelEncoder().fit_transform(X['enrolled_university'])
X['education_level']=LabelEncoder().fit_transform(X['education_level'])
X['major_discipline']=LabelEncoder().fit_transform(X['major_discipline'])
X['experience']=LabelEncoder().fit_transform(X['experience'])
X['company_size']=LabelEncoder().fit_transform(X['company_size'])
X['company_type']=LabelEncoder().fit_transform(X['company_type'])
X['last_new_job']=LabelEncoder().fit_transform(X['last_new_job'])

In [ ]:
X.info()

In [ ]:
X.drop('target',axis=1,inplace=True)

In [ ]:
for col in X.columns:
    print(col)
    sns.boxplot(x=X[col],data=X)
    plt.show()  
    
# boxplot for outliers

In [ ]:
def boxoutlier(var):
    for x in var.iloc[:,0:1].columns :
        
        Q1=var[x].quantile(0.25)
        Q3=var[x].quantile(0.75)
        IQR=Q3-Q1
        Lower = Q1-(1.5*IQR)
        Upper = Q3+(1.5*IQR)
#         print(Lower, Upper)

        var.loc[:,x]=np.where(var[x].values > Upper,Upper,var[x].values)
        var.loc[:,x]=np.where(var[x].values < Lower,Lower,var[x].values)
        
    return var
X=boxoutlier(X)

# treatment of outliers

In [ ]:
for col in X.columns:
    print(col)
    sns.boxplot(x=X[col],data=X)
    plt.show()

# confirming for existence of outliers

In [ ]:
X['training_hours']=np.log(X['training_hours'].astype(int))

# data spread was high thats why log() is used 

In [ ]:
for col in X.columns:
    print(col)
    sns.boxplot(x=X[col],data=X)
    plt.show()

In [ ]:
def boxoutlier(var):
    for x in var.iloc[:,-1:].columns :
        
        print(var.iloc[:,-1:-2].columns)
        Q1=var[x].quantile(0.25)
        Q3=var[x].quantile(0.75)
        IQR=Q3-Q1
        Lower = Q1-(1.5*IQR)
        Upper = Q3+(1.5*IQR)
#         print(Lower, Upper)

        var.loc[:,x]=np.where(var[x].values > Upper,Upper,var[x].values)
        var.loc[:,x]=np.where(var[x].values < Lower,Lower,var[x].values)
        
    return var
tr2=boxoutlier(X)
# df2=boxoutlier(x_train['training hours'])

In [ ]:
for col in tr2.columns:
    print(col)
    sns.boxplot(x=tr2[col],data=tr2)
    plt.show()

In [ ]:
X=tr2

In [ ]:
X.info()
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=4 )

In [ ]:
display(X_train.head(),y_train.head(),'Testing Data',X_test.head(),y_test.head())

In [ ]:
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
clf=KNeighborsClassifier(n_neighbors=17,metric= 'manhattan')

In [ ]:
## checking for value of k_neighbors
from sklearn.metrics import accuracy_score
error_rate = []
for i in range(1,20):
    knn1 = KNeighborsClassifier(n_neighbors=i)
    knn1.fit(X_train,y_train)
    pred_i = knn1.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))
plt.figure(figsize=(10,6))
plt.plot(range(1,20),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=5)   ## plot() used to plot the points on the canvas
plt.title('Error Rate vs. K Value')
plt.xlabel('K')  ## Labelling for X_axis
plt.ylabel('Error Rate')  ## Labelling for X_axis
y_pred = knn1.predict(X_test) ## predict() 
print(accuracy_score(y_test, y_pred)*100) ## checking accuracy score
y_pred  

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
# Checking accuracy of Model

In [ ]:
pred_test=clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred_test)

##Accuracy of Testing model

In [ ]:
pred_train=clf.predict(X_train)
from sklearn.metrics import accuracy_score
accuracy_score(y_train,pred_train)

##Accuracy of Training model

In [ ]:
# Accuracy of Training model and Accuracy of Testing model is almost same that means
# our model is good its neither underfit nor overfit

In [ ]:
# Probability

In [ ]:
clf.classes_

In [ ]:
prob=clf.predict_proba(X_test)
prob
df1=pd.DataFrame(prob,columns=clf.classes_,index=X_test.index)
df1.head()
df1['pred']=pred_test
df1['y_test']=y_test

df1['updated']=df1[0.0].apply(lambda x :1 if x<0.3 else 0)  ## 0.3 is threshold value 
# df1['updated'].value_counts()
df1.head()

# predict_proba gives you the probabilities for the target (0 and 1 in your case) in array form.
# The number of probabilities for each row is equal to the number of categories in target 
# variable (2 in your case).

In [ ]:
df1['pred'].value_counts()

In [ ]:
# Evaluation of Model using Classification Matrix,Confusion Matrix

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,pred_test))

In [ ]:
print(confusion_matrix(y_test,pred_test))

In [ ]:
print(classification_report(y_train,pred_train))

In [ ]:
print(confusion_matrix(y_train,pred_train))